In [2]:
import sys
!{sys.executable} -m pip install git+https://github.com/nottombrown/imagenet_stubs
sys.path.append("../..")

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow_hub as thub
import tensorflow.keras.backend as K

from lib.data_utils import *

from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod
import art

  Cloning https://github.com/nottombrown/imagenet_stubs to /private/var/folders/fj/_ghtrmrs4sq75w4yjfz9ncv00000gn/T/pip-req-build-_jrj_os4
  Running command git clone -q https://github.com/nottombrown/imagenet_stubs /private/var/folders/fj/_ghtrmrs4sq75w4yjfz9ncv00000gn/T/pip-req-build-_jrj_os4
  Created wheel for imagenet-stubs: filename=imagenet_stubs-0.0.7-py3-none-any.whl size=794838 sha256=4de12e03a1203ff5caa1346c0266b4640b554ef230ea89a0aa916298f4b7d1f4
  Stored in directory: /private/var/folders/fj/_ghtrmrs4sq75w4yjfz9ncv00000gn/T/pip-ephem-wheel-cache-o5t5iqsp/wheels/64/e6/a3/53e4a13354406428112f1ae250f746f879b8e9ab55397fa2d1
Successfully built imagenet-stubs


ModuleNotFoundError: No module named 'lib'

In [2]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [3]:
# Load the raw CIFAR-10 data
cifar10_dir = 'lib/datasets/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(cifar10_dir)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [5]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [6]:
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

ganeval_module = thub.Module(model_url)
# gan_eval_layer = thub.KerasLayer(model_url)

In [7]:
print(tf.__version__)

1.15.0


In [8]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [10]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [31]:
# Another ge_cifar model with more layers

K.clear_session()
#model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

ganeval_module_new = thub.Module(model_url)
# gan_eval_layer = thub.KerasLayer(model_url)


ge_cifar_new_clf = tf.keras.Sequential()
ge_cifar_new_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
ge_cifar_new_clf.add(thub.KerasLayer(ganeval_module_new))
ge_cifar_new_clf.add(tf.keras.layers.Activation('softmax'))
ge_cifar_new_clf.add(tf.keras.layers.Activation('relu'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [32]:
ge_cifar_new_clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [35]:
ge_cifar_new_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

In [36]:
ge_cifar_new_clf.evaluate(testd, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0017 - acc: 1.0000


[0.00166802722774446, 1.0]

# Testing the retrieved hub model by evaluating on train data
Should give us 100% or something close to that

In [11]:
ge_cifar_clf.evaluate(testd, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0017 - acc: 1.0000


[0.001668029203079641, 1.0]

# Testing a sample attack

In [12]:
from art.attacks import FastGradientMethod

classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.4)

In [13]:
x_test_adv = attack_fgsm.generate(testd.copy())

In [14]:
ge_cifar_clf.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 6.9334 - acc: 0.1020


[6.933382461547851, 0.102]

# Define the substitute classifier for our model

In [15]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [16]:
def build_substitute_model():
    model = tf.keras.Sequential( )
    model.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
    model.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
    model.add( MaxPooling2D( pool_size=(2, 2) ) )
    model.add( Flatten( ) )
    model.add( Dense( 128, activation='relu' ) )
    model.add( Dense(10, activation='softmax' ) )
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=[ 'accuracy' ] )
    
    return model

In [20]:
sub_clf = build_substitute_model()

sub_clf.summary( )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2097280   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,117,962
Trainable params: 2,117,962
Non-trainable params: 0
____________________________________________

In [21]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [22]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=15)

(7500, 32, 32, 3) (7500, 10)


In [23]:
cloud_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
extracted_art_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=False)

In [25]:
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets

attack_CCC = CopycatCNN(classifier=cloud_clf, 
                        batch_size_fit=8, 
                        batch_size_query=8,
                        nb_epochs=20,
                        nb_stolen=1000)

In [26]:
attack_CCC.extract(partial_x_train, partial_y_train, thieved_classifier=extracted_art_clf)

Epoch 1/20
125/125 [==============================] - 3s 20ms/step - loss: 2.1702 - acc: 0.2100
Epoch 2/20
125/125 [==============================] - 2s 15ms/step - loss: 1.7718 - acc: 0.3440
Epoch 3/20
125/125 [==============================] - 2s 16ms/step - loss: 1.4329 - acc: 0.5080
Epoch 4/20
125/125 [==============================] - 2s 17ms/step - loss: 1.0283 - acc: 0.6740
Epoch 5/20
125/125 [==============================] - 2s 16ms/step - loss: 0.7143 - acc: 0.7770
Epoch 6/20
125/125 [==============================] - 2s 16ms/step - loss: 0.4322 - acc: 0.8720
Epoch 7/20
125/125 [==============================] - 2s 17ms/step - loss: 0.3511 - acc: 0.9030
Epoch 8/20
125/125 [==============================] - 2s 17ms/step - loss: 0.2162 - acc: 0.9350
Epoch 9/20
125/125 [==============================] - 2s 16ms/step - loss: 0.1237 - acc: 0.9650
Epoch 10/20
125/125 [==============================] - 2s 16ms/step - loss: 0.0920 - acc: 0.9770
Epoch 11/20
125/125 [==================

art.classifiers.keras.KerasClassifier(model=<tensorflow.python.keras.engine.sequential.Sequential object at 0x14317a828>, use_logits=False, channel_index=3, clip_values=(0, 1), preprocessing_defences=None, postprocessing_defences=None, preprocessing=(0, 1), input_layer=0, output_layer=0)

In [28]:
cloud_clf._model.evaluate(x_train_adv, y_train_adv)
extracted_art_clf._model.evaluate(x_train_adv, y_train_adv)

1000/1000 [==============================] - 0s 350us/sample - loss: 3.6477 - acc: 0.3800


[3.6476968822479248, 0.38]

In [29]:
cloud_clf._model.evaluate(x_test_adv, y_test_adv)
extracted_art_clf._model.evaluate(x_test_adv, y_test_adv)

1000/1000 [==============================] - 0s 254us/sample - loss: 3.4995 - acc: 0.3790


[3.4995197010040284, 0.379]

# Attacks

# Pixel Attack

In [38]:
from art.attacks.evasion import PixelAttack
pixelA_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [39]:
pixel_attack = PixelAttack(classifier = pixelA_clf, 
                         th=None, es=1, targeted=False, verbose=False)

In [40]:
x_test_adv_pixel = pixel_attack.generate(np.array(x_train_adv[:10].copy()))

/Library/anaconda3/lib/python3.7/site-packages/art/attacks/evasion/pixel_threshold.py:1136: RuntimeWarning: divide by zero encountered in double_scalars
  and self.callback(self._scale_parameters(self.population[0]), convergence=self.tol / convergence)


In [41]:
ge_cifar_new_clf.evaluate(x_test_adv_pixel, np.array(y_train_adv[:10]))

10/10 [==============================] - 0s 4ms/sample - loss: 16.1181 - acc: 0.0000e+00


[16.11809539794922, 0.0]

# Threshold Attack

In [42]:
from art.attacks.evasion import ThresholdAttack
thresholdA_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [132]:
threshold_attack = ThresholdAttack(classifier = thresholdA_clf, es=1, targeted=False, verbose=False)
# classifier, th=None, es=0, targeted=False, verbose=False

In [133]:
x_test_adv_threshold = threshold_attack.generate(x_train_adv[:10].copy())

KeyboardInterrupt: 

# HopSkipJump Attack - check

In [45]:
from art.attacks.evasion import HopSkipJump
hopSkipJump_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [46]:
hopSkipJump_attack = HopSkipJump(classifier=hopSkipJump_clf, targeted=False, norm=2,
                                 max_iter=50, max_eval=10000, init_eval=10, init_size=10)
# classifier, targeted=False, norm=2, max_iter=50, max_eval=10000, init_eval=100, init_size=100

In [47]:
x_test_adv_hopSkipJump = hopSkipJump_attack.generate(x_train_adv[:10].copy())

In [48]:
ge_cifar_new_clf.evaluate(x_test_adv_hopSkipJump, y_train_adv[:10])

10/10 [==============================] - 0s 4ms/sample - loss: 1.0986 - acc: 0.0000e+00


[1.0985829830169678, 0.0]

# Projected Gradient Descent (PGD)

In [49]:
from art.attacks.evasion import ProjectedGradientDescent
projectedGradientDescent_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [50]:
pgd_attack = ProjectedGradientDescent(classifier=projectedGradientDescent_clf, eps=0.4)
# classifier, norm=inf, eps=0.3, eps_step=0.1, max_iter=100, targeted=False, num_random_init=0, batch_size=1, random_eps=False

In [51]:
x_test_adv_pgd = pgd_attack.generate(x_train_adv[:10].copy())

In [52]:
ge_cifar_new_clf.evaluate(x_test_adv_pgd, y_train_adv[:10])

10/10 [==============================] - 0s 4ms/sample - loss: 16.1181 - acc: 0.0000e+00


[16.11809539794922, 0.0]

# NewtonFool

In [53]:
from art.attacks.evasion import NewtonFool
newtonFool_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [54]:
newtonFool_attack = NewtonFool(classifier=newtonFool_clf, max_iter=100, eta=0.01, batch_size=1)
# classifier, max_iter=100, eta=0.01, batch_size=1

In [55]:
x_test_adv_newtonFool = newtonFool_attack.generate(x_train_adv[:100].copy())

In [57]:
ge_cifar_new_clf.evaluate(x_test_adv_newtonFool, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 2.4086 - acc: 0.0100


[2.408556547164917, 0.01]

# Elastic Net Attack

In [58]:
from art.attacks.evasion import ElasticNet
elasticNet_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [129]:
elasticNet_attack = ElasticNet(classifier=elasticNet_clf, decision_rule='EN')
# classifier, confidence=0.0, targeted=False, learning_rate=0.01, binary_search_steps=9, max_iter=100, beta=0.001, initial_const=0.001, batch_size=1, decision_rule='EN'

In [130]:
x_test_adv_elasticNet = elasticNet_attack.generate(x_train_adv[:10].copy())

In [131]:
ge_cifar_new_clf.evaluate(x_test_adv_elasticNet, y_train_adv[:10])

10/10 [==============================] - 0s 4ms/sample - loss: 3.3999 - acc: 0.0000e+00


[3.399890899658203, 0.0]

# Spatial transformation attack

In [61]:
from art.attacks.evasion import SpatialTransformation
spatialT_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [62]:
spatialT_attack = SpatialTransformation(classifier=spatialT_clf, max_translation=0.0, num_translations=1, max_rotation=0.0, num_rotations=1)
# classifier, max_translation=0.0, num_translations=1, max_rotation=0.0, num_rotations=1

In [64]:
x_test_adv_elasticNet = spatialT_attack.generate(x_train_adv[:10].copy())

In [65]:
ge_cifar_new_clf.evaluate(x_test_adv_elasticNet, y_train_adv[:10])

10/10 [==============================] - 0s 4ms/sample - loss: 1.7695e-04 - acc: 1.0000


[0.00017694880079943687, 1.0]

# Zeroth-order optimization attack

In [67]:
from art.attacks.evasion import ZooAttack
zoo_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [68]:
zoo_attack = ZooAttack(classifier=zoo_clf,confidence=0.0, targeted=False, learning_rate=0.01, max_iter=10, 
                              binary_search_steps=1, initial_const=0.001, abort_early=True, use_resize=True, 
                              use_importance=True, nb_parallel=128, batch_size=1, variable_h=0.0001)
# classifier, confidence=0.0, targeted=False, learning_rate=0.01, max_iter=10, binary_search_steps=1, initial_const=0.001, abort_early=True, use_resize=True, use_importance=True, nb_parallel=128, batch_size=1, variable_h=0.0001

In [69]:
x_test_adv_zoo = zoo_attack.generate(x_train_adv[:10].copy())

In [70]:
ge_cifar_new_clf.evaluate(x_test_adv_zoo, y_train_adv[:10])

10/10 [==============================] - 0s 4ms/sample - loss: 1.7695e-04 - acc: 1.0000


[0.00017694880079943687, 1.0]

# Carlini and Wagner L_inf Attack

In [71]:
from art.attacks.evasion import CarliniLInfMethod
carliniLInf_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [72]:
carliniLInf_attack = CarliniLInfMethod(classifier=carliniLInf_clf, confidence=0.0, targeted=False, 
                                       learning_rate=0.01, max_iter=10, max_halving=5, max_doubling=5, 
                                       eps=0.3, batch_size=128)
# classifier, confidence=0.0, targeted=False, learning_rate=0.01, max_iter=10, max_halving=5, max_doubling=5, eps=0.3, batch_size=128

In [74]:
x_test_adv_carliniLInf = carliniLInf_attack.generate(x_train_adv[:100].copy())

In [76]:
ge_cifar_new_clf.evaluate(x_test_adv_carliniLInf, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 3.5337 - acc: 0.0500


[3.53366548538208, 0.05]

# Jacobian Saliency Map Attack

In [77]:
from art.attacks.evasion import SaliencyMapMethod
saliencyMap_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [78]:
saliency_attack = SaliencyMapMethod(classifier=saliencyMap_clf, theta=0.1, gamma=1.0, batch_size=50)
# classifier, eps=0.3, eps_step=0.1, max_iter=100, targeted=False, batch_size=1

In [79]:
x_test_adv_saliency = saliency_attack.generate(x_train_adv[:50].copy())

In [81]:
ge_cifar_new_clf.evaluate(x_test_adv_saliency, y_train_adv[:50]) 

50/50 [==============================] - 0s 3ms/sample - loss: 1.0477 - acc: 0.0000e+00


[1.0477229070663452, 0.0]

# Universal perturbation Moosavi

In [83]:
from art.attacks.evasion import UniversalPerturbation
universalPerturbation_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [84]:
universalPerturbation_attack = UniversalPerturbation(classifier=universalPerturbation_clf, attacker='deepfool', attacker_params=None, delta=0.2, max_iter=20, eps=10.0, norm=2)
# classifier, eps=0.3, eps_step=0.1, max_iter=100, targeted=False, batch_size=1

In [85]:
x_test_adv_universalPerturbation = universalPerturbation_attack.generate(x_train_adv[:100].copy())

art.attacks.evasion.deepfool.DeepFool


In [87]:
ge_cifar_new_clf.evaluate(x_test_adv_universalPerturbation, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 7.0494 - acc: 0.0400


[7.049377346038819, 0.04]

# DeepFool attack

In [89]:
from art.attacks.evasion import DeepFool
deepFool_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [90]:
deepFool_attack = DeepFool(classifier=deepFool_clf, max_iter=100, epsilon=1e-06, nb_grads=10, batch_size=1)
# classifier, eps=0.3, eps_step=0.1, max_iter=100, targeted=False, batch_size=1

In [91]:
x_test_adv_deepFool = deepFool_attack.generate(x_train_adv[:100].copy())

In [96]:
ge_cifar_new_clf.evaluate(x_test_adv_deepFool, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 4.9037 - acc: 0.0200


[4.903667392730713, 0.02]

# Virtual Adversarial Method

In [92]:
from art.attacks.evasion import VirtualAdversarialMethod
virtualAdversarial_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [93]:
virtualAdversarial_attack = DeepFool(classifier=virtualAdversarial_clf, max_iter=100, nb_grads=10, batch_size=100)
# classifier, max_iter=100, epsilon=1e-06, nb_grads=10, batch_size=1

In [94]:
x_test_adv_virtualAdv = virtualAdversarial_attack.generate(x_train_adv[:100].copy())

In [97]:
ge_cifar_new_clf.evaluate(x_test_adv_virtualAdv, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 4.9037 - acc: 0.0200


[4.903668823242188, 0.02]

# Fast Gradient Method Attack

In [98]:
from art.attacks.evasion import FastGradientMethod
fgm_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [99]:
fgm_attack = FastGradientMethod(classifier=fgm_clf, eps=0.3, eps_step=0.1, targeted=False, 
                                num_random_init=0, batch_size=1, minimal=False)
# classifier, norm=inf, eps=0.3, eps_step=0.1, targeted=False, num_random_init=0, batch_size=1, minimal=False

In [100]:
x_test_adv_fgm = fgm_attack.generate(x_train_adv[:100].copy())

In [102]:
ge_cifar_new_clf.evaluate(x_test_adv_fgm, y_train_adv[:100])

100/100 [==============================] - 0s 3ms/sample - loss: 3.9604 - acc: 0.0900


[3.960428466796875, 0.09]

In [134]:
predict_fgsm = ge_cifar_new_clf.predict(x_test_adv[:10])

# Query-Efficient Black-Box Attack

In [ ]:
from art.wrappers import QueryEfficientBBGradientEstimation
queryEffBlackBox_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
qeBlackBox_attack = QueryEfficientBBGradientEstimation(classifier=queryEffBlackBox_clf, num_basis=0, sigma=0, round_samples=0)
# classifier, num_basis, sigma, round_samples=0

In [ ]:
qeBlackBox_attack.fit(testd[:10], testl[:10])

In [ ]:
qeBlackBox_lossGradient = qeBlackBox_attack.loss_gradient(testd[:10], testl[:10])

In [ ]:
qeBB_prediction = qeBlackBox_attack.predict(x_test[:10])

# Defences

# Gaussian Data Augmentation Defence

In [109]:
from art.defences.preprocessor import GaussianAugmentation
## do something

In [110]:
gaussian_defence = GaussianAugmentation(sigma=1.0, augmentation=True, ratio=1.0, clip_values=None, apply_fit=True, apply_predict=False)

In [113]:
gaus_aug_data = gaussian_defence.__call__(x=x_test_adv[:50], y=y_train_adv[:50])
gaus_aug_data

(array([[[[ 6.07843137e-01,  6.11764706e-01,  5.84313725e-01],
          [ 6.54901961e-01,  6.90196078e-01,  7.33333333e-01],
          [ 6.90196078e-01,  7.01960784e-01,  7.56862745e-01],
          ...,
          [ 7.88235294e-01,  7.68627451e-01,  8.19607843e-01],
          [ 7.92156863e-01,  7.92156863e-01,  8.31372549e-01],
          [ 7.52941176e-01,  7.17647059e-01,  6.70588235e-01]],
 
         [[ 6.00000000e-01,  6.07843137e-01,  6.15686275e-01],
          [ 6.39215686e-01,  7.01960784e-01,  8.00000000e-01],
          [ 6.70588235e-01,  7.21568627e-01,  8.43137255e-01],
          ...,
          [ 7.92156863e-01,  8.15686275e-01,  8.70588235e-01],
          [ 7.96078431e-01,  8.43137255e-01,  8.90196078e-01],
          [ 7.41176471e-01,  7.45098039e-01,  7.17647059e-01]],
 
         [[ 6.07843137e-01,  6.03921569e-01,  6.00000000e-01],
          [ 6.27450980e-01,  6.98039216e-01,  7.88235294e-01],
          [ 6.58823529e-01,  7.25490196e-01,  8.35294118e-01],
          ...,
    

In [114]:
est_grad = gaussian_defence.estimate_gradient(x_test_adv[:50], grad = 0.001)
est_grad

0.001

In [ ]:
gaussian_defence

# Feature Squeezing

In [115]:
from art.defences.preprocessor import FeatureSqueezing

In [119]:
featureSqueezing_defence = FeatureSqueezing(clip_values=(0,255), bit_depth=8, apply_fit=True, apply_predict=False)
# clip_values = (min,max), bit_depth=8, apply_fit=False, apply_predict=True
featureSqueezing_defence

In [120]:
est_grad_featSqz = featureSqueezing_defence.estimate_gradient(x_test_adv[:50], grad = 0.001)
est_grad_featSqz

0.001

# High Confidence Defence

In [122]:
from art.defences.postprocessor import HighConfidence

In [124]:
highConf_defence = HighConfidence(cutoff=0.25, apply_fit=False, apply_predict=True)
highConf_defence

In [125]:
highConf_defence.__call__(x_test_adv[:50])

array([[[[0.60784314, 0.61176471, 0.58431373],
         [0.65490196, 0.69019608, 0.73333333],
         [0.69019608, 0.70196078, 0.75686275],
         ...,
         [0.78823529, 0.76862745, 0.81960784],
         [0.79215686, 0.79215686, 0.83137255],
         [0.75294118, 0.71764706, 0.67058824]],

        [[0.6       , 0.60784314, 0.61568627],
         [0.63921569, 0.70196078, 0.8       ],
         [0.67058824, 0.72156863, 0.84313725],
         ...,
         [0.79215686, 0.81568627, 0.87058824],
         [0.79607843, 0.84313725, 0.89019608],
         [0.74117647, 0.74509804, 0.71764706]],

        [[0.60784314, 0.60392157, 0.6       ],
         [0.62745098, 0.69803922, 0.78823529],
         [0.65882353, 0.7254902 , 0.83529412],
         ...,
         [0.80784314, 0.81568627, 0.84313725],
         [0.8       , 0.84313725, 0.87843137],
         [0.74117647, 0.74901961, 0.72156863]],

        ...,

        [[0.34901961, 0.30980392, 0.        ],
         [0.        , 0.        , 0.        ]

# Rounding/Rounded Defence

In [126]:
from art.defences.postprocessor import Rounded

In [127]:
rounded_defence = Rounded(decimals=3, apply_fit=False, apply_predict=True)

In [128]:
rounded_defence.__call__(x_test_adv[:50])

array([[[[0.608, 0.612, 0.584],
         [0.655, 0.69 , 0.733],
         [0.69 , 0.702, 0.757],
         ...,
         [0.788, 0.769, 0.82 ],
         [0.792, 0.792, 0.831],
         [0.753, 0.718, 0.671]],

        [[0.6  , 0.608, 0.616],
         [0.639, 0.702, 0.8  ],
         [0.671, 0.722, 0.843],
         ...,
         [0.792, 0.816, 0.871],
         [0.796, 0.843, 0.89 ],
         [0.741, 0.745, 0.718]],

        [[0.608, 0.604, 0.6  ],
         [0.627, 0.698, 0.788],
         [0.659, 0.725, 0.835],
         ...,
         [0.808, 0.816, 0.843],
         [0.8  , 0.843, 0.878],
         [0.741, 0.749, 0.722]],

        ...,

        [[0.349, 0.31 , 0.243],
         [0.224, 0.208, 0.204],
         [0.239, 0.239, 0.231],
         ...,
         [0.349, 0.467, 0.396],
         [0.231, 0.31 , 0.243],
         [0.263, 0.29 , 0.196]],

        [[0.349, 0.322, 0.243],
         [0.235, 0.227, 0.208],
         [0.247, 0.259, 0.227],
         ...,
         [0.224, 0.259, 0.227],
         [0.

# Virtual Adversarial Training Defence

In [103]:
from art.defences.trainer import AdversarialTrainer
advTrainer_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [104]:
adversarialTrainer_defence = AdversarialTrainer(classifier=advTrainer_clf, attacks=fgm_attack, ratio=0.5)

In [105]:
adversarialTrainer_defence.fit(x_train_adv[:50].copy(), y_train_adv[:50].copy())

1/1 [==============================] - 0s 139ms/step - loss: 3.1076 - acc: 0.4800


In [106]:
at_predictions = adversarialTrainer_defence.predict(x_test_adv[:50].copy())